In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchtext
from konlpy.tag import Kkma
from torchtext.data import Field,Iterator,Example, TabularDataset

http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

In [4]:
torchtext.__version__

'0.2.1'

<img src="image/torchtext.png">

## 1. Field 선언 

Field는 데이터의 전처리 파이프라인을 정의하는 클래스

http://torchtext.readthedocs.io/en/latest/data.html#field

### 문장 - 클래스 

In [40]:
tagger = Kkma()
tokenize = tagger.morphs

TEXT = Field(tokenize=tokenize,use_vocab=True,lower=True, include_lengths=True, batch_first=True) 
LABEL = Field(sequential=False,unk_token=None, use_vocab=True)

## 2. 데이터셋 로드

In [41]:
train_data, test_data = TabularDataset.splits(
                                   path="data/", # 데이터가 있는 root 경로
                                   train='train.txt', validation="test.txt",
                                   format='tsv', # \t로 구분
                                   #skip_header=True, # 헤더가 있다면 스킵
                                   fields=[('TEXT',TEXT),('LABEL',LABEL)])

In [42]:
one_example = train_data.examples[0]

In [43]:
one_example.TEXT

['배고프', '다', '밥', '주', '어']

In [44]:
one_example.LABEL

'FOOD'

## 3. Vocabulary 구축

In [45]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [46]:
len(TEXT.vocab)

119

In [47]:
TEXT.vocab.itos

['<unk>',
 '<pad>',
 '어',
 '하',
 '추천',
 'ㄴ',
 '주',
 '?',
 '는',
 '먹',
 '좀',
 '집',
 '고',
 '맛',
 '뭐',
 '영화',
 '다',
 '싶',
 '거',
 '근처',
 '보',
 '보여주',
 '아',
 '알려주',
 '음식',
 '이',
 '재밌',
 'ㅇ',
 'ㅊ',
 'ㅎ',
 '가',
 '나',
 '냐',
 '드라마',
 '밥',
 '배고프',
 '배달',
 '볼만',
 '삼겹살',
 '어디',
 '없',
 '예능',
 '요즘',
 '유',
 '을까',
 '있',
 '지',
 '튜버',
 '하이라이트',
 '해',
 'ㅂ니다',
 'ㅠㅜㅠ',
 '가장',
 '갈',
 '강남',
 '개운',
 '검색',
 '겁',
 '경기',
 '고등',
 '과',
 '귀신',
 '금요일',
 '기',
 '나오',
 '너',
 '니',
 '다시',
 '대',
 '대박',
 '도',
 '동영상',
 '되',
 '떡',
 '떡볶이',
 '랩',
 '러',
 '막',
 '만',
 '만나',
 '만하',
 '말',
 '맛있',
 '뭔',
 '바',
 '방송',
 '보내주',
 '서',
 '스릴',
 '슬프',
 '신',
 '어서',
 '어제',
 '에',
 '역',
 '엽',
 '영상',
 '올림픽',
 '웃기',
 '을',
 '이번',
 '인생',
 '일정',
 '잼지',
 '점',
 '줄거리',
 '중계',
 '지금',
 '최신',
 '축',
 '케이블',
 '코미디',
 '포장',
 '푸',
 '함께',
 '핫',
 '해장',
 '화',
 '황금빛']

In [48]:
LABEL.vocab.itos

['FOOD', 'MEDIA']

## 4. Iterator 선언 

In [49]:
train_data

In [50]:
test_data

In [51]:
# make iterator for splits
train_iter, test_iter = Iterator.splits(
    (train_data, test_data), batch_size=3, device=-1, # device -1 : cpu, device 0 : 남는 gpu
    sort_key=lambda x: len(x.TEXT),sort_within_batch=True,repeat=False) # x.TEXT 길이 기준으로 정렬

In [52]:
TEXT.vocab.itos[1]

'<pad>'

In [53]:
for batch in train_iter:
    print(batch.TEXT)
    print(batch.LABEL)
    break

(Variable containing:
 38   9  12  17   2
 38  69  11  23   2
 34   6   2   1   1
[torch.LongTensor of size 3x5]
, 
 5
 5
 3
[torch.LongTensor of size 3]
)
Variable containing:
 0
 0
 0
[torch.LongTensor of size 3]



## 5. Modeling and Training 

In [54]:
class EmbedClassifier(nn.Module):
    def __init__(self,vocab_size,embedding_size,output_size):
        super(EmbedClassifier,self).__init__()
        
        # 각 단어의 임베딩을 평균해서 문장 단위의 임베딩 표현
        self.sentence_embed = nn.EmbeddingBag(vocab_size,embedding_size)
        self.linear = nn.Linear(embedding_size,output_size)
    
    def forward(self,inputs):
        outputs = self.sentence_embed(inputs)
        outputs = self.linear(outputs)
        return outputs

In [59]:
STEP=50
LR = 0.1

model = EmbedClassifier(len(TEXT.vocab),30,2)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=LR)

In [60]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(train_iter):
        inputs,lengths = batch.TEXT
        targets = batch.LABEL
        model.zero_grad()
        preds = model(inputs)
        loss = loss_function(preds,targets)
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
    if step % 10==0:
        print(np.mean(losses))
        losses=[]

0.7632251190287727
0.4217357113957405
0.2671015997018133
0.16162016548748528
0.10611527439739023


## 6. Test

파라미터를 학습하기에 데이터수가 너무 적음..!!

In [61]:
for test in test_data.examples:
    input, length = TEXT.numericalize(([test.TEXT],[len(test.TEXT)]),train=False,device=-1)
    pred = model(input)
    pred = pred.max(1)[1]
    print(pred.data[0],test.LABEL)

0 FOOD
0 FOOD
0 MEDIA
0 MEDIA


### numericalize 

문장 ==> 인덱스에 맞는 Variable(LongTensor)로 변환

In [62]:
TEXT.numericalize(([test.TEXT],[len(test.TEXT)]),train=False,device=-1)

(Variable containing:
     0     0     0     0    20    12     0
 [torch.LongTensor of size 1x7], 
  7
 [torch.LongTensor of size 1])